In [ ]:
import pandas as pd
import numpy as np
import os
import re
import time
import datetime
import gc
import string
import re
import seaborn as sns 
import matplotlib.pyplot as plt 
import math

from decimal import Decimal, getcontext

# --------------------------------------------

output_dir = r"E:\Cristian\1. Molecular Dinamics\2. Python Projects\3. Jupiter Code\7. Nose_Hoover Fixed\2. N_1000\4. Q+_Q+\10. T_0.3"

# Construir las rutas de los archivos basadas en output_dir
Sigma_Charge_data_file = os.path.join(output_dir, "91. Sigma_&_Charge_MD.csv")
Potencial_data_file = os.path.join(output_dir, "92. Potencial_MD.csv")

MD_Sigma_Charge_DF = pd.read_csv(Sigma_Charge_data_file,sep='\t')
MD_Potencial_DF = pd.read_csv(Potencial_data_file,sep='\t')

# Renombrar la columna 'Edad' a 'Años'
MD_Potencial_DF.rename(columns={'U': 'u'}, inplace=True)

# --------------------------------------------

simulation_data_file = os.path.join(output_dir, "Simulation_Values.csv")
Simulation_Values = pd.read_csv(simulation_data_file)

Q_p = float(Simulation_Values.loc[Simulation_Values['Variable'] == 'Q_p', 'Valor'].iloc[0])
Q_n = float(Simulation_Values.loc[Simulation_Values['Variable'] == 'Q_n', 'Valor'].iloc[0])
m = float(Simulation_Values.loc[Simulation_Values['Variable'] == 'm', 'Valor'].iloc[0])
N = float(Simulation_Values.loc[Simulation_Values['Variable'] == 'N', 'Valor'].iloc[0])
kB = float(Simulation_Values.loc[Simulation_Values['Variable'] == 'kB', 'Valor'].iloc[0])
Temp_Target = float(Simulation_Values.loc[Simulation_Values['Variable'] == 'Temp_Target', 'Valor'].iloc[0])
e0 = float(Simulation_Values.loc[Simulation_Values['Variable'] == 'e0', 'Valor'].iloc[0])
R = float(Simulation_Values.loc[Simulation_Values['Variable'] == 'R', 'Valor'].iloc[0])


r_2 = float(Simulation_Values.loc[Simulation_Values['Variable'] == 'r_2', 'Valor'].iloc[0])
r_1 = float(Simulation_Values.loc[Simulation_Values['Variable'] == 'r_1', 'Valor'].iloc[0])
r_G = float(Simulation_Values.loc[Simulation_Values['Variable'] == 'r_G', 'Valor'].iloc[0])
Mass = float(Simulation_Values.loc[Simulation_Values['Variable'] == 'm', 'Valor'].iloc[0])
IDEAL = bool(Simulation_Values.loc[Simulation_Values['Variable'] == 'IDEAL', 'Valor'].iloc[0])

Lim_Inf = 2500 * 000
Lim_Sup = 2500 * 400


In [ ]:

#------------------------ Lectura MoM Method Data ------------------------

MoM_Sigma_Charge_data_file = os.path.join(output_dir, "sigmaTwoSheets.dat")
MoM_Sigma_Charge_DF = pd.read_csv(MoM_Sigma_Charge_data_file,sep='\t')

MoM_Potencial_004_data_file = os.path.join(output_dir, "PhiZ0_004.dat")
MoM_Potencial_04_data_file  = os.path.join(output_dir, "PhiZ0_04.dat")
MoM_Potencial_08_data_file  = os.path.join(output_dir, "PhiZ0_08.dat")
MoM_Potencial_2_data_file   = os.path.join(output_dir, "PhiZ2.dat")
MoM_Potencial_4_data_file   = os.path.join(output_dir, "PhiZ4.dat")
MoM_Potencial_8_data_file   = os.path.join(output_dir, "PhiZ8.dat")
MoM_Potencial_40_data_file  = os.path.join(output_dir, "PhiZ40.dat")

MoM_Potencial_004 = pd.read_csv(MoM_Potencial_004_data_file,sep='\t')
MoM_Potencial_04  = pd.read_csv(MoM_Potencial_04_data_file ,sep='\t')
MoM_Potencial_08  = pd.read_csv(MoM_Potencial_08_data_file ,sep='\t')
MoM_Potencial_2   = pd.read_csv(MoM_Potencial_2_data_file  ,sep='\t')
MoM_Potencial_4   = pd.read_csv(MoM_Potencial_4_data_file  ,sep='\t')
MoM_Potencial_8   = pd.read_csv(MoM_Potencial_8_data_file  ,sep='\t')
MoM_Potencial_40  = pd.read_csv(MoM_Potencial_40_data_file ,sep='\t')

MoM_Potencial = MoM_Potencial_004.merge(MoM_Potencial_04, on='u', how='left') \
                                 .merge(MoM_Potencial_08, on='u', how='left') \
                                 .merge(MoM_Potencial_2 , on='u', how='left') \
                                 .merge(MoM_Potencial_4 , on='u', how='left') \
                                 .merge(MoM_Potencial_8 , on='u', how='left') \
                                 .merge(MoM_Potencial_40, on='u', how='left')

MoM_Charge_data_file = os.path.join(output_dir, "chargeTwoSheets.dat")
MoM_Charge = pd.read_csv(MoM_Charge_data_file,sep='\t')

del MoM_Sigma_Charge_data_file,MoM_Potencial_004_data_file,MoM_Potencial_04_data_file,MoM_Potencial_08_data_file
del MoM_Potencial_2_data_file,MoM_Potencial_4_data_file,MoM_Potencial_8_data_file,MoM_Potencial_40_data_file
del MoM_Potencial_004,MoM_Potencial_04,MoM_Potencial_08,MoM_Potencial_2,MoM_Potencial_4,MoM_Potencial_8,MoM_Potencial_40
del MoM_Charge_data_file,simulation_data_file,Potencial_data_file

In [ ]:

plt.plot(MD_Sigma_Charge_DF['u/R'], MD_Sigma_Charge_DF['Sigma'],linestyle='-', marker='x', markersize=8, linewidth=1,label='MD')
plt.plot(MoM_Sigma_Charge_DF['u/R'], MoM_Sigma_Charge_DF['Sigma'],linestyle='-', marker='o', markersize=4, linewidth=1,label='MoM')

# Configurar el título, etiquetas de ejes y leyenda
plt.title('Charge Density MD vs MoM')
plt.xlabel('u/R')
plt.ylabel('Sigma')
plt.legend(loc='best')  # Agregar una leyenda con la ubicación óptima
# Mostrar la gráfica
plt.grid(True)
plt.savefig(os.path.join(output_dir, "Sigma_MoM_vs_MD.png"))
# plt.show()


In [ ]:

plt.plot(MD_Sigma_Charge_DF['u/R'], MD_Sigma_Charge_DF['q/Q'],linestyle='-', marker='x', markersize=8, linewidth=1,label='MD')
plt.plot(MoM_Charge['u/R'], MoM_Charge['q/Q'],linestyle='-', marker='o', markersize=4, linewidth=1,label='MoM')

# Configurar el título, etiquetas de ejes y leyenda
plt.title('Charge MD vs MoM')
plt.xlabel('u/R')
plt.ylabel('q/Q')
plt.legend(loc='best')  # Agregar una leyenda con la ubicación óptima
# Mostrar la gráfica
plt.grid(True)
plt.savefig(os.path.join(output_dir, "Charge_MoM_vs_MD.png"))
# plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 12))

MD_Col = list(MD_Potencial_DF)
MD_Col.remove("u")

for i in MD_Col:
    columna = i  # Nombre de la columna en MD_Potencial_DF
    plt.plot(MD_Potencial_DF['u']/r_1, MD_Potencial_DF[columna],linestyle='-', marker='x', markersize=8, linewidth=1)

MoM_Col = list(MoM_Potencial)
MoM_Col.remove("u")

# Crear gráficas para MoM_Potencial
for i in MoM_Col:
    columna = i  # Nombre de la columna en MoM_Potencial
    plt.plot(MoM_Potencial['u']/r_1, MoM_Potencial[columna],linestyle='-', marker='o', markersize=4, linewidth=1)

# Configurar el título, etiquetas de ejes y leyenda
plt.title('Potencial MD vs MoM')
plt.xlabel('u/R')
plt.ylabel('Potencial')
# plt.legend(loc='best')  # Agregar una leyenda con la ubicación óptima

# Agregar una etiqueta para 'MD'
plt.annotate('-o- = MoM', xy=(0.82, 0.9), xycoords='axes fraction', fontsize=24)

# Agregar una etiqueta para 'MoM'
plt.annotate('-x- = MoM', xy=(0.82, 0.95), xycoords='axes fraction', fontsize=24)

plt.grid(True)
plt.savefig(os.path.join(output_dir, "Potencial_MoM_vs_MD.png"))
# plt.show()
